#구글 플레이 앱스토어에서 스크랩 된 데이터
#각 앱 (행)에는 카테고리, 등급, 크기 등에 대한 값이 있습니다.
#Play 스토어 앱 데이터는 앱 제작 비즈니스를 성공으로 이끄는 엄청난 잠재력을 가지고 있습니다. 
#개발자가 Android 시장에서 작업하고 점령 할 수 있도록 실행 가능한 통찰력을 얻을 수 있습니다!

1) App :- Name of the App
2) Category :- Category under which the App falls.
3) Rating :- Application's rating on playstore
4) Reviews :- Number of reviews of the App.
5) Size :- Size of the App.
6) Install :- Number of Installs of the App
7) Type :- If the App is free/paid
8) Price :- Price of the app (0 if it is Free)
9) Content Rating :- Appropiate Target Audience of the App.
10) Genres:- Genre under which the App falls.
11) Last Updated :- Date when the App was last updated
12) Current Ver :- Current Version of the Application
13) Android Ver :- Minimum Android Version required to run the App

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

playapp = pd.read_csv("C:/Users/SAMSUNG/Desktop/googleplaystore.csv")
print(playapp.shape)

In [ ]:
playapp.head(5)

In [ ]:
playapp.tail(5)

#앱, 카테고리,타입,콘텐트 레이팅, 장르는 문자형 변수
#rating,reviews,size,installs,price,current ver, android ver는 숫자형
#last updated는 날짜형변수
#리뷰-숫자형으로 바꾸기, size는 m을 제거하고 숫자형으로 바꾸기, installs는 +제거하고 숫자형으로 바꾸기
#price,current ver, android ver 숫자형으로 바꾸기
#last updated는 날짜형으로 바꾸기 혹은 숫자형으로

In [ ]:
playapp.info()

In [ ]:
playapp.isnull().sum()

In [ ]:
#rating이 결측값이 많으므로 median으로 대체하기, type은 drop

In [ ]:
playapp.describe()

In [ ]:
playapp['Price'].unique()

# Data Cleaning

In [ ]:
# 결측값 대체와 제거

In [ ]:
playapp['Rating'] = playapp['Rating'].fillna(playapp['Rating'].median())

In [ ]:
playapp = playapp[pd.notnull(playapp['Type'])]
playapp = playapp[pd.notnull(playapp['Content Rating'])]
playapp = playapp[pd.notnull(playapp['Current Ver'])]
playapp = playapp[pd.notnull(playapp['Android Ver'])]

In [ ]:
playapp.isnull().sum()

In [ ]:
#카테고리 변수에 대한 전처리 과정

In [ ]:
playapp['Category'].unique()

In [ ]:
import re
import seaborn as sns
import time

In [ ]:
# 현재 버전 변수를 number.number의 포맷으로 변환하는 과정 to simplify the data
# all non numerical values & unicode charachters를 제거해야 한다. 

replaces = [u'\u00AE', u'\u2013', u'\u00C3', u'\u00E3', u'\u00B3', '[', ']', "'"]
for i in replaces:
    playapp['Current Ver'] = playapp['Current Ver'].astype(str).apply(lambda x : x.replace(i, ''))

regex = [r'[-+|/:/;(_)@]', r'\s+', r'[A-Za-z]+']
for j in regex:
    playapp['Current Ver'] = playapp['Current Ver'].astype(str).apply(lambda x : re.sub(j, '0', x))

playapp['Current Ver'] = playapp['Current Ver'].astype(str).apply(lambda x : x.replace('.', ',',1).replace('.', '').replace(',', '.',1)).astype(float)


# Encoding

In [ ]:
#앱, 카테고리,타입,콘텐트 레이팅, 장르는 문자형 변수
#rating,reviews,size,installs,price,current ver, android ver는 숫자형
#last updated는 날짜형변수
#리뷰-숫자형으로 바꾸기, size는 m을 제거하고 숫자형으로 바꾸기, installs는 +제거하고 숫자형으로 바꾸기
#price,current ver, android ver 숫자형으로 바꾸기
#last updated는 날짜형으로 바꾸기 혹은 숫자형으로

In [ ]:
#엔코딩 종류는 레이블엔코딩과 원핫엔코딩이 있음
#레이블 엔코딩은 각범주별 숫자를 부여하기 때문에 선형 모형을 쓸 경우 숫자로 인식하여 적절하지 않음
#나중에 선형 모형 몇개도 적합할 것이므로 원핫인코딩만 사용함

In [ ]:
from sklearn import preprocessing

In [ ]:
# 앱 변수 encoding
playapp['App'] = pd.get_dummies(playapp['App'],drop_first=True)
playapp['Category']=pd.get_dummies(playapp['Category'],drop_first=True)
playapp['Type']=pd.get_dummies(playapp['Type'],drop_first=True)
playapp['Content Rating']=pd.get_dummies(playapp['Content Rating'],drop_first=True)
playapp['Genres']=pd.get_dummies(playapp['Genres'],drop_first=True)

In [ ]:
# Last Updated (날짜형변수 변환) encoding - 문자열을 datetime으로 변환
import datetime
playapp['Last Updated'] = playapp['Last Updated'].apply(lambda x : time.mktime(datetime.datetime.strptime(x, '%B %d, %Y').timetuple()))

# Price cleaning-unique함수로 살펴본 바 있음
playapp['Price'] = playapp['Price'].apply(lambda x : x.strip('$'))

# Installs cleaning
playapp['Installs'] = playapp['Installs'].apply(lambda x : x.strip('+').replace(',', ''))

# Convert kbytes to Mbytes
k_indices = playapp['Size'].loc[playapp['Size'].str.contains('k')].index.tolist()
converter = pd.DataFrame(playapp.loc[k_indices, 'Size'].apply(lambda x: x.strip('k')).astype(float).apply(lambda x: x / 1024).apply(lambda x: round(x, 3)).astype(str))
playapp.loc[k_indices,'Size'] = converter

# Size cleaning-M붙어있는거 제거하고 기기에 따라 다양한 행도 0으로 맞춰준 후 float형으로 타입변환
playapp['Size'] = playapp['Size'].apply(lambda x: x.strip('M'))
playapp[playapp['Size'] == 'Varies with device'] = 0
playapp['Size'] = playapp['Size'].astype(float)

# Modeling

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso

from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_squared_error,mean_absolute_error,r2_score

In [ ]:
features = ['App','Category','Reviews','Size','Installs','Type','Price','Content Rating','Genres','Last Updated','Current Ver']

X = playapp[features]
y = playapp['Rating']

#Python 리스트에 새로운 원소를 추가하는 방법에는
#append(x)와 extend(iterable)가 있고 두 함수의 차이점을 알아보겠습니다.(참고로 insert(i, x)함수도 있으며 위치 i에 x를 추가합니다)
#append는 리스트 자체를 추가하고 extend는 원소를 추가함.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=10)

1.Decision Tree Regression

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeRegressor(criterion='mae', max_depth=5, min_samples_leaf=5, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

2.Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators = 200, n_jobs=-1, random_state=10)
rf.fit(X_train, y_train)
rf.score(X_test,y_test)

In [ ]:
3.Multiple Linear Regression 

In [ ]:
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(X_train,y_train)
mlr.score(X_test,y_test)                                  

#R^2<0인 경우, 편차제곱<오차제곱인 상황으로
#내가 만든 regression model이 그냥 mean value로 예측하는 모델보다 예측성능이 더 낮은 비정상적인 경우를 의미

4.Ridge model

In [ ]:
ridge = Ridge()
ridge.fit(X_train,y_train)
ridge.score(X_test,y_test)

5.Lasso model

In [ ]:
lasso = Lasso()
lasso.fit(X_train,y_train)
lasso.score(X_test,y_test)

In [ ]:
# 결정트리 - 91.88
# 랜덤포레스트 - 92.47
# 다중 회귀 - 오류
# 릿지 - 91.77
# 라쏘 - 91.75
# 랜포>트리>릿지>라쏘